## The Data
This is a dataset of food products listing the ingredients and nutritional facts of more than 300,000 foods from over 150 countries in the world. The data is meant to be free to the public to help users decode food labels, and make better food choices in general. The dataset contains more than 300,000 rows over 163 columns. However, as we will see there are a lot of missing or obviously incorrect values.<br/>
Generalities:

- fields that end with _t are dates in the UNIX timestamp format (number of seconds since Jan 1st 1970)
- fields that end with _datetime are dates in the iso8601 format: yyyy-mm-ddThh:mn:ssZ
- fields that end with _tags are comma separated list of tags (e.g. categories_tags is the set of normalized tags computer from the categories field)
- fields that end with a language 2 letter code (e.g. fr for French) is the set of tags in that language
- fields that end with _100g correspond to the amount of a nutriment (in g, or kJ for energy) for 100 g or 100 ml of product
- fields that end with _serving correspond to the amount of a nutriment (in g, or kJ for energy) for 1 serving of the product


# Nettoyer et Filtrer les Features et Produits
## 1.1. Collecter et Préparer les Données Brutes
### 1. Charger les données dans un DataFrame Pandas


In [ ]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
import pandas as pd

import missingno as msno

## 1.1. Collecter et Préparer les Données Brutes
### 1. Charger les données dans un DataFrame Pandas

In [ ]:
def init():
    print('Initialisation du projet...')
    filepath ='datas/openfoodfacts.csv'
    target ='origins'
    selected = ['pnns_groups_1','pnns_groups_2','nutrition_grade_fr']
    data = pd.read_csv(f"{filepath}", sep="\t", encoding="utf-8", on_bad_lines="skip", low_memory=False)
    return filepath,target,selected

In [ ]:
def select_file():
    print("charger le nouveau fichier")
    root =tk.Tk()
    root.withdraw()

    filepath = filedialog.askopenfilename(
        title="Choisissez votre fichier csv",
        filetypes=(("Fichiers CSV","*.csv"), ("Tous les fichiers","*.*"))
    )
    print("chargement en cours...")
    root.destroy()

    if filepath:
        try:
            data = pd.read_csv(f"{filepath}", sep="\t", encoding="utf-8", on_bad_lines="skip", low_memory=False)
            print(f"Fichier chargé avec succès depuis le chemin: {filepath}")
            return data
        except Exeption as e:
            print("Erreur lors du chargement du fichier:",e)
    else:
        print("Aucun fichier sélectionné.")

In [ ]:
data.shape

Il s'agit d'un gros dataset de plus de 320 000 ligne et 160 colonnes

In [ ]:
info = data.info()

In [ ]:
data.head()

## 1.2. Identifier et Sélectionner les Features Pertinentes
### 1.Lister les Features (Colonnes) :

In [ ]:
def feature_list(data):
    features = data.columns
    print(f"Total features:", len(features))
    print(f"Features:", features)


### 2.Séparer les Features Quantitatives et Qualitatives :

In [ ]:
def split_features(data):
    quantitative_features = data.select_dtypes(include=['float64','int64'])
    qualitative_features = data.select_dtypes(include=['object'])
    return quantitative_features,qualitative_features

print('Quantitative Features\n',quantitative_features.columns )
print('Qualitative Features\n',qualitative_features.columns )

## 1.3. Choisir une Cible

### 1.Choisir une Feature Cible avec Moins de 50% de Valeurs Présentes :

In [ ]:
def target_choice(qualitative_features):
    # Calcule le pourcentage de valeurs manquantes pour chaque colonne
    missing_percentages = data.isnull().mean() *100
    #Filtrer les features ayant moins de 50% de valeurs manquantes.
    features_with_less_than_50_missing = missing_percentages[missing_percentages > 50].index
    potential_targets = [feature for feature in qualitative_features if feature in features_with_less_than_50_missing]

    print(f'{len(potential_targets)} cibles potentiels / {len(qualitative_features.columns)} features. \nSelectionner votre cible dans la liste suivantes :')
    for index, feature in enumerate(potential_targets):
        #num_unique_item = data[feature].nunique()
        nb_fill_each_value= data[feature].count()
        val_null= data[feature].isna().sum()
        percentage_fill_each_value = (nb_fill_each_value / len(qualitative_features) * 100).round(2)
        print(f"{index+1}: {feature}: {percentage_fill_each_value}% des valeurs présentes soit {nb_fill_each_value} valeurs")
        #print(f"{index+1}: {feature}: {num_unique_item} valeurs unique / Pourcentage de valeurs présentes: {percentage_fill_each_value}%")
    try:
        selected_index = int(input('Entrer l\'index de votre cible:'))
        target = potential_targets[selected_index-1]
        print("Feature cible selectionnée:", target)
    except IndexError:
        print("Invalid index. Please restart the selection process.")
    except ValueError:
        print("Invalid input. Please enter a numeric index.")
    return target

In [ ]:
filepath,target,selected=init() #initialisation du fichier local avec la première selection de paramètres
print('Paramètre par défaut:\n')
print(f'chemin du fichier:\n --> {filepath}\n')
print(f'Cible de l\'analyse:\n --> {target}\n')
print(f'Features de l\'analyse:\n --> {selected}\n')
default_params=input('Souhaitez-vous changer les paramètres par défault?\n Oui / Non')

if default_params.lower() == "oui":
    
    filepath_params=input('Souhaitez-vous changer de fichier?\n Oui / Non')
    if filepath_params.lower()=='oui':
        data = select_file()  
        feature_list(data)
        quantitative_features, qualitative_features = split_features(data)
    
    target_params=input(f'Souhaitez-vous changer la cible d\'analye? {target}\n Oui / Non')
    if target_params.lower()=='oui':
        target = target_choice(qualitative_features)
        filling_rate_display(data_cleaned,df_without_target)
        select_features_with_more_than_50(df_without_target)
    print(f"cible d'analyse: {target}")
    data_clean = clean_row_without_target(data,target)
    df_without_target = split_dataset_from_target(data_cleaned,target)
    filling_rate_display(data_cleaned,df_without_target)
    
    selected_params=input(f'Souhaitez-vous changer de Features? actuellement {selected}\n Oui / Non')
    if selected_params.lower()=='oui':
        selected_features=select_features_with_more_than_50(df_without_target)
        selected =features_selection(selected,selected_features)
        print(f'Nouvelles Features selectionnées:{selected}')
    
else:
    feature_list(data)
    quantitative_features, qualitative_features = split_features(data)
    #print(f'split-----> {qualitative_features}')
    #target_choice(qualitative_features)
    data_clean = clean_row_without_target(data,target)
    df_without_target = split_dataset_from_target(data_cleaned,target)
    filling_rate_display(data_cleaned,df_without_target)
    #def cleanAndFilterProductFeatures:


In [ ]:
#result=data[target].dropna().unique()
#df_result=pd.DataFrame(result, columns=[target])
#print(f'Détail de notre features cibles: {target}')
#df_result

### Supprimer les Lignes (Produits) Sans Valeur Cible :


In [ ]:
data[target].isna().sum()

In [ ]:
def clean_row_without_target(data,target):
    data_nan = data[data[target].isna()]
    data_cleaned = data.dropna(subset=[target])
    result = data_cleaned.shape
    return data_cleaned
#data_nan.shape

### 3.Séparer la Cible du Reste du Dataset :


In [ ]:
def split_dataset_from_target(data_cleaned,target):
    df_without_target = data_cleaned.drop(columns=[target])
    df_with_target=data_cleaned[target]
    return df_without_target

## 1.4. Afficher les Taux de Remplissage des Features

In [ ]:
def filling_rate_display(data_cleaned,df_without_target):
    filling_rates = data_cleaned.notnull().mean()*100
    sorted_value=df_without_target.count().sort_values(ascending=True)
    sorted_df =df_without_target[sorted_value.index]
    msno.bar(sorted_df)

## 1.5. Sélectionner des Features Intéressantes

### Sélectionner les Features avec Plus de 50% de Valeurs Présentes :

In [ ]:
def select_features_with_more_than_50(df_without_target):
    df_without_target=df_without_target.notnull().mean()*100
    selected_features = df_without_target[df_without_target>50].index
    selected_features = pd.DataFrame(selected_features,columns=['Features'])
    print('Feature que vous pouvez selectionner:\n ')
    for index ,row in selected_features.iterrows():
        print(f'{index+1} : {row["Features"]}')
    return selected_features
#selected_features.columns=['Features']

In [ ]:
def features_selection(selected,selected_features):
    nb_element = int(input('Combien de feature voulez-vous utiliser (recommandé 3)'))
    print(f'Selectionnez 1 à {nb_element} éléments:\n')
    #first_element = input(f'Entrez le numéro de votre premier choix pour terminer:')
    #first_element = selected_features.iloc(first_element)
    old_selected = selected
    selected = [] # Pour stocker les features choisis
    while len(selected) < nb_element:
        if len(selected)<nb_element:
            choice_index= input(f"Entrez le numéro de votre choix numéro {len(selected)+1} ou tapez fin pour garder le choix initial")
            if choice_index.lower()=='fin':
                if len(selected)==0:
                    selected = old_selected
                else:
                    return selected
            else:
                try:
                    choice_index= int(choice_index)-1
                    if 0<= choice_index <len(selected_features):
                        feature = selected_features.iloc[choice_index]['Features']
                        if feature not in selected:
                            selected.append(feature)
                            print(f'Vous avez selectionné {feature}')
                            print(f'Features selectionnées:{selected}')
                        else:
                            print("Vous avez déjà sélectionné cette feature. Veuillez choisir une autre.")
                    else:
                        print('Numéro de choix invalide, veuillez entrer un numéro valide.')
                except ValueError:
                    print("Entrée invalide, veuillez entrer un numéro entier.")
    return selected